In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install optuna==2.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.2/308.2 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.2/147.2 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 KB 12.4 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=fef5582cc0ad51d6f5f00ff6e0c939d8caa455642ac15c6d34e92f0af8a289da
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


## 1. Tabular Data & Modules Load



In [ ]:
import os, sys
from pathlib import Path

# Set up path
try:
    base_path = Path(__file__).resolve().parent.parent
except:
    %cd '/content/drive/MyDrive/Colab Notebooks/kaggle/Player-Contact-Detection/tutorial'
    base_path = Path('__file__').resolve().parent.parent

module_path = base_path / 'module'
data_path = base_path / 'data'

# Append system path 
sys.path.append(str(module_path))

/content/drive/MyDrive/Colab Notebooks/kaggle/Player-Contact-Detection/tutorial


In [132]:
# Import data analysis libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost.sklearn import XGBClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef
import tqdm
from ensemble import BinaryCalssifier
# from jtlearn import ImbSampler

In [ ]:
# Load datasets
tr_baseline_helmets = pd.read_csv(data_path / 'train_baseline_helmets.csv')
tr_labels = pd.read_csv(data_path / 'train_labels.csv')
tr_player_tracking = pd.read_csv(data_path / 'train_player_tracking.csv')
tr_video_metadata = pd.read_csv(data_path / 'train_video_metadata.csv')

te_baseline_helmets = pd.read_csv(data_path / 'test_baseline_helmets.csv')
te_player_tracking = pd.read_csv(data_path / 'test_player_tracking.csv')
te_video_metadata = pd.read_csv(data_path / 'test_video_metadata.csv')

submission = pd.read_csv(data_path / 'sample_submission.csv')

In [ ]:
def inplace(func):
    def wrapper(data, inplace=False):                           # 호출할 함수를 감싸는 함수
        if inplace:
            func(data, inplace)
        else:
            df = data.copy()
            func(df, inplace)
            return df
    return wrapper                           # wrapper 함수 반환

## contact id 펼치는 함수
@inplace
def expand_contact_id(df, inplace: bool=False):
    """
    Splits out contact_id into seperate columns.
    """
    df['game_play'] = df['contact_id'].str[:12]
    df[['step', 'nfl_player_id_1', 'nfl_player_id_2']] = df.contact_id.str.split('_', expand=True).iloc[:, 2:]
    df['step'] = df['step'].astype('int')
    df['nfl_player_id_1'] = df['nfl_player_id_1'].astype('int')
    df['group_id'] = df.apply(lambda x: '_'.join([x.game_play, x.nfl_player_id_1, x.nfl_player_id_2]),
                              axis=1)

    temp = df['contact']
    df.drop(columns=['contact'], inplace=True)
    df['contact'] = temp
    return df

## contact_id 만드는 함수
@inplace
def create_contact_id(df, inplace: bool=False):
    # Create contact ids
    cols = list(df)
    if df.nfl_player_id_2.dtype == 'str':
        df.loc[:, 'contact_id'] = (
            df.loc[:, 'game_play']
            + '_'
            + df.loc[:, 'step'].astype('str')
            + '_'
            + df.loc[:, 'nfl_player_id_1'].astype('str')
            + '_'
            + df.loc[:, 'nfl_player_id_2']
        )
    else:
        df.loc[:, 'contact_id'] = (
            df.loc[:, 'game_play']
            + '_'
            + df.loc[:, 'step'].astype('str')
            + '_'
            + df.loc[:, 'nfl_player_id_1'].astype('str')
            + '_'
            + df.loc[:, 'nfl_player_id_2'].astype('str')
        )

    temp = df[cols]
    df.drop(columns=cols, inplace=True)
    df[cols] = temp
    del temp
    return df

## contact id 만드는 함수
@inplace
def create_group_id(df, inplace: bool=False):
    # Create contact ids
    cols = list(df)
    if df.nfl_player_id_2.dtype == 'str':
        df.loc[:, 'group_id'] = (
            df.loc[:, 'game_play']
            + '_'
            + df.loc[:, 'nfl_player_id_1'].astype('str')
            + '_'
            + df.loc[:, 'nfl_player_id_2']
        )
    else:
        df.loc[:, 'group_id'] = (
            df.loc[:, 'game_play']
            + '_'
            + df.loc[:, 'nfl_player_id_1'].astype('str')
            + '_'
            + df.loc[:, 'nfl_player_id_2'].astype('str')
        )

    temp = df[cols]
    df.drop(columns=cols, inplace=True)
    df[cols] = temp
    del temp
    return df

In [ ]:
te_labels = submission.copy()
expand_contact_id(te_labels, inplace=True)
te_labels = te_labels[['contact_id', 'group_id', 'game_play', 'step',
                       'nfl_player_id_1', 'nfl_player_id_2', 'contact']]
display(te_labels)

create_group_id(tr_labels, inplace=True)
tr_labels.drop(columns=['datetime'], inplace=True)
tr_labels = tr_labels[['contact_id', 'group_id', 'game_play', 'step',
                       'nfl_player_id_1', 'nfl_player_id_2', 'contact']]
display(te_labels)

,contact_id,group_id,game_play,step,nfl_player_id_1,nfl_player_id_2,contact
0,58168_003392_0_38590_43854,58168_003392_38590_43854,58168_003392,0,38590,43854,0
1,58168_003392_0_38590_41257,58168_003392_38590_41257,58168_003392,0,38590,41257,0
2,58168_003392_0_38590_41944,58168_003392_38590_41944,58168_003392,0,38590,41944,0
3,58168_003392_0_38590_42386,58168_003392_38590_42386,58168_003392,0,38590,42386,0
4,58168_003392_0_38590_47944,58168_003392_38590_47944,58168_003392,0,38590,47944,0
...,...,...,...,...,...,...,...
49583,58172_003247_125_40656_G,58172_003247_40656_G,58172_003247,125,40656,G,0
49584,58172_003247_125_52521_G,58172_003247_52521_G,58172_003247,125,52521,G,0
49585,58172_003247_125_52939_G,58172_003247_52939_G,58172_003247,125,52939,G,0
49586,58172_003247_125_39008_G,58172_003247_39008_G,58172_003247,125,39008,G,0


,contact_id,group_id,game_play,step,nfl_player_id_1,nfl_player_id_2,contact
0,58168_003392_0_38590_43854,58168_003392_38590_43854,58168_003392,0,38590,43854,0
1,58168_003392_0_38590_41257,58168_003392_38590_41257,58168_003392,0,38590,41257,0
2,58168_003392_0_38590_41944,58168_003392_38590_41944,58168_003392,0,38590,41944,0
3,58168_003392_0_38590_42386,58168_003392_38590_42386,58168_003392,0,38590,42386,0
4,58168_003392_0_38590_47944,58168_003392_38590_47944,58168_003392,0,38590,47944,0
...,...,...,...,...,...,...,...
49583,58172_003247_125_40656_G,58172_003247_40656_G,58172_003247,125,40656,G,0
49584,58172_003247_125_52521_G,58172_003247_52521_G,58172_003247,125,52521,G,0
49585,58172_003247_125_52939_G,58172_003247_52939_G,58172_003247,125,52939,G,0
49586,58172_003247_125_39008_G,58172_003247_39008_G,58172_003247,125,39008,G,0


In [ ]:
# 야드 to 미터 변환
yard2meter = 0.9144
convert_columns = ['x_position', 'y_position', 'speed', 'distance', 'acceleration', 'sa']
tr_player_tracking[convert_columns] = tr_player_tracking[convert_columns] * yard2meter
te_player_tracking[convert_columns] = te_player_tracking[convert_columns] * yard2meter
display(tr_player_tracking)
display(te_player_tracking)

,game_play,game_key,play_id,nfl_player_id,datetime,step,team,position,jersey_number,x_position,y_position,speed,distance,direction,orientation,acceleration,sa
0,58580_001136,58580,1136,44830,2021-10-10T21:08:20.900Z,-108,away,CB,22,56.317896,38.953440,1.014984,0.100584,320.33,263.93,0.649224,-0.585216
1,58580_001136,58580,1136,47800,2021-10-10T21:08:20.900Z,-108,away,DE,97,54.388512,24.515064,0.210312,0.009144,346.84,247.16,1.179576,0.822960
2,58580_001136,58580,1136,52444,2021-10-10T21:08:20.900Z,-108,away,FS,29,66.010536,28.767024,0.557784,0.054864,11.77,247.69,0.576072,-0.301752
3,58580_001136,58580,1136,46206,2021-10-10T21:08:20.900Z,-108,home,TE,86,52.459128,20.226528,0.338328,0.036576,127.85,63.63,0.630936,0.566928
4,58580_001136,58580,1136,52663,2021-10-10T21:08:20.900Z,-108,away,ILB,48,57.835800,25.146000,0.466344,0.045720,183.62,253.71,0.283464,0.283464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353048,58575_003081,58575,3081,48476,2021-10-10T19:22:06.800Z,140,away,CB,40,66.092832,47.365920,1.453896,0.155448,345.36,342.68,0.448056,-0.374904
1353049,58575_003081,58575,3081,44887,2021-10-10T19:22:06.800Z,140,away,ILB,45,68.360544,49.834800,0.896112,0.091440,342.39,354.46,0.576072,-0.219456
1353050,58575_003081,58575,3081,44174,2021-10-10T19:22:06.800Z,140,away,ILB,49,68.159376,50.977800,1.024128,0.100584,352.79,349.20,0.576072,0.420624
1353051,58575_003081,58575,3081,45217,2021-10-10T19:22:06.800Z,140,away,TE,82,73.554336,44.595288,1.764792,0.182880,340.78,346.51,0.484632,-0.466344


,game_play,game_key,play_id,nfl_player_id,datetime,step,team,position,jersey_number,x_position,y_position,speed,distance,direction,orientation,acceleration,sa
0,58172_003247,58172,3247,41937,2020-09-13T19:30:20.200Z,-272,home,MLB,57,58.777632,10.323576,4.407408,0.420624,20.74,12.43,2.862072,2.743200
1,58172_003247,58172,3247,45345,2020-09-13T19:30:20.200Z,-272,away,ILB,50,69.521832,33.595056,2.880360,0.292608,179.52,184.91,1.709928,-1.655064
2,58172_003247,58172,3247,46205,2020-09-13T19:30:20.200Z,-272,home,DE,98,60.021216,14.392656,1.645920,0.182880,6.73,339.85,0.630936,-0.621792
3,58172_003247,58172,3247,43406,2020-09-13T19:30:20.200Z,-272,home,WR,14,33.238440,13.661136,1.655064,0.164592,330.50,344.03,0.566928,0.356616
4,58172_003247,58172,3247,48233,2020-09-13T19:30:20.200Z,-272,away,RB,45,69.585840,29.397960,3.072384,0.310896,166.59,180.56,0.804672,-0.795528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14867,58168_003392,58168,3392,43395,2020-09-11T03:01:57.300Z,92,away,FS,23,46.661832,12.463272,1.554480,0.155448,21.01,18.32,0.320040,-0.182880
14868,58168_003392,58168,3392,39947,2020-09-11T03:01:57.300Z,92,home,T,72,39.072312,16.934688,1.088136,0.118872,331.22,332.97,0.530352,-0.512064
14869,58168_003392,58168,3392,44822,2020-09-11T03:01:57.300Z,92,home,QB,15,32.013144,17.080992,1.353312,0.137160,165.22,162.66,0.347472,-0.064008
14870,58168_003392,58168,3392,38590,2020-09-11T03:01:57.300Z,92,home,G,70,39.977568,16.678656,1.408176,0.146304,341.08,336.00,0.374904,-0.374904


In [ ]:
def convert_angular_direction(theta):
    angle = 90 - theta
    if angle < 0:
        angle = 360 + angle
    return angle

# 각도 변환
angle_columns = ['direction', 'orientation']
tr_player_tracking[angle_columns] = tr_player_tracking[angle_columns].applymap(convert_angular_direction)
te_player_tracking[angle_columns] = te_player_tracking[angle_columns].applymap(convert_angular_direction)

## 2. Data preprocessing

In [ ]:
## 지면 충돌
tr_ground_contact = tr_labels[tr_labels.nfl_player_id_2 == 'G']
te_ground_contact = te_labels[te_labels.nfl_player_id_2 == 'G']

## 선수간 충돌
tr_players_contact = tr_labels[tr_labels.nfl_player_id_2 != 'G']
te_players_contact = te_labels[te_labels.nfl_player_id_2 != 'G']

tr_players_contact.nfl_player_id_2 = tr_players_contact.nfl_player_id_2.astype(int)
te_players_contact.nfl_player_id_2 = te_players_contact.nfl_player_id_2.astype(int)

tr_ground_contact

,contact_id,group_id,game_play,step,nfl_player_id_1,nfl_player_id_2,contact
231,58168_003392_0_38590_G,58168_003392_38590_G,58168_003392,0,38590,G,0
232,58168_003392_0_43854_G,58168_003392_43854_G,58168_003392,0,43854,G,0
233,58168_003392_0_41257_G,58168_003392_41257_G,58168_003392,0,41257,G,0
234,58168_003392_0_41944_G,58168_003392_41944_G,58168_003392,0,41944,G,0
235,58168_003392_0_42386_G,58168_003392_42386_G,58168_003392,0,42386,G,0
...,...,...,...,...,...,...,...
4721613,58582_003121_91_48220_G,58582_003121_48220_G,58582_003121,91,48220,G,0
4721614,58582_003121_91_47906_G,58582_003121_47906_G,58582_003121,91,47906,G,0
4721615,58582_003121_91_38557_G,58582_003121_38557_G,58582_003121,91,38557,G,0
4721616,58582_003121_91_47872_G,58582_003121_47872_G,58582_003121,91,47872,G,0


In [ ]:
te_players_contact

,contact_id,group_id,game_play,step,nfl_player_id_1,nfl_player_id_2,contact
0,58168_003392_0_38590_43854,58168_003392_38590_43854,58168_003392,0,38590,43854,0
1,58168_003392_0_38590_41257,58168_003392_38590_41257,58168_003392,0,38590,41257,0
2,58168_003392_0_38590_41944,58168_003392_38590_41944,58168_003392,0,38590,41944,0
3,58168_003392_0_38590_42386,58168_003392_38590_42386,58168_003392,0,38590,42386,0
4,58168_003392_0_38590_47944,58168_003392_38590_47944,58168_003392,0,38590,47944,0
...,...,...,...,...,...,...,...
49561,58172_003247_125_47912_48241,58172_003247_47912_48241,58172_003247,125,47912,48241,0
49562,58172_003247_125_47912_47920,58172_003247_47912_47920,58172_003247,125,47912,47920,0
49563,58172_003247_125_47912_48335,58172_003247_47912_48335,58172_003247,125,47912,48335,0
49564,58172_003247_125_47912_52521,58172_003247_47912_52521,58172_003247,125,47912,52521,0


### Physical Analysis

In [ ]:
def acc_perpendicular(direct: float, acc: float, sa: float) -> float:
    res = np.sqrt(acc ** 2 - sa ** 2)
    if direct < 0:
        res = -res
    return res

def vectorization(data: pd.DataFrame) -> pd.DataFrame:
    deg2rad = np.pi / 180
    df = data.copy()

    df['x_vel'] = df.apply(lambda x: x.speed * np.cos(x.direction * deg2rad),
                           axis=1)
    df['y_vel'] = df.apply(lambda x: x.speed * np.sin(x.direction * deg2rad),
                           axis=1)

    df['sign_perpendicular'] = df.direction.diff().replace({0: np.nan})
    df['sign_perpendicular'] = df['sign_perpendicular'].fillna(method='ffill').fillna(method='bfill')
    df['sa_perpendicular'] = df.apply(lambda x: acc_perpendicular(x.sign_perpendicular, x.acceleration, x.sa),
                                axis=1)

    df['x_acc'] = df.apply(lambda x: x.sa * np.cos(x.direction * deg2rad), axis=1) + \
                  df.apply(lambda x: x.sa_perpendicular * np.cos((90 + x.direction) * deg2rad), axis=1)
    df['y_acc'] = df.apply(lambda x: x.sa * np.sin(x.direction * deg2rad), axis=1) + \
                  df.apply(lambda x: x.sa_perpendicular * np.sin((90 + x.direction) * deg2rad), axis=1)
    
    df = df.drop(columns=['sign_perpendicular', 'sa_perpendicular'])
    return df

In [ ]:
def make_full_tracking(tracking):
    tracking_ls = []
    for game_play in tracking.game_play.unique():
        game_df = tracking[tracking.game_play == game_play]
        temp_ls = []

        for player_id in game_df.nfl_player_id.unique():
            game_player_df = game_df[game_df.nfl_player_id == player_id]
            temp_ls.append(vectorization(game_player_df))

        temp_df = pd.concat(temp_ls, axis=0)
        temp_df = temp_df.sort_values(by='step', ascending=True)
        tracking_ls.append(temp_df)

    result = pd.concat(tracking_ls, axis=0)
    return result.sort_index()

train_player_tracking = make_full_tracking(tr_player_tracking)
test_player_tracking = make_full_tracking(te_player_tracking)

In [ ]:
train_player_tracking

NameError: ignored

In [ ]:
train_player_tracking.drop(columns=['Unnamed: 0'])

,game_play,game_key,play_id,nfl_player_id,datetime,step,team,position,jersey_number,x_position,...,speed,distance,direction,orientation,acceleration,sa,x_vel,y_vel,x_acc,y_acc
0,58580_001136,58580,1136,44830,2021-10-10T21:08:20.900Z,-108,away,CB,22,56.317896,...,1.014984,0.100584,129.67,186.07,0.649224,-0.585216,-0.647930,0.781268,0.157213,-0.629901
1,58580_001136,58580,1136,47800,2021-10-10T21:08:20.900Z,-108,away,DE,97,54.388512,...,0.210312,0.009144,103.16,202.84,1.179576,0.822960,-0.047882,0.204789,0.635509,0.993745
2,58580_001136,58580,1136,52444,2021-10-10T21:08:20.900Z,-108,away,FS,29,66.010536,...,0.557784,0.054864,78.23,202.31,0.576072,-0.301752,0.113779,0.546056,-0.541953,-0.195309
3,58580_001136,58580,1136,46206,2021-10-10T21:08:20.900Z,-108,home,TE,86,52.459128,...,0.338328,0.036576,322.15,26.37,0.630936,0.566928,0.267151,-0.207597,0.277754,-0.566510
4,58580_001136,58580,1136,52663,2021-10-10T21:08:20.900Z,-108,away,ILB,48,57.835800,...,0.466344,0.045720,266.38,196.29,0.283464,0.283464,-0.029444,-0.465414,-0.017898,-0.282898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353048,58575_003081,58575,3081,48476,2021-10-10T19:22:06.800Z,140,away,CB,40,66.092832,...,1.453896,0.155448,104.64,107.32,0.448056,-0.374904,-0.367465,1.406692,-0.142638,-0.424745
1353049,58575_003081,58575,3081,44887,2021-10-10T19:22:06.800Z,140,away,ILB,45,68.360544,...,0.896112,0.091440,107.61,95.54,0.576072,-0.219456,-0.271106,0.854118,0.574066,-0.048031
1353050,58575_003081,58575,3081,44174,2021-10-10T19:22:06.800Z,140,away,ILB,49,68.159376,...,1.024128,0.100584,97.21,100.80,0.576072,0.420624,-0.128535,1.016030,-0.443296,0.367897
1353051,58575_003081,58575,3081,45217,2021-10-10T19:22:06.800Z,140,away,TE,82,73.554336,...,1.764792,0.182880,109.22,103.49,0.484632,-0.466344,-0.580963,1.666425,0.028993,-0.483764


In [ ]:
train_player_tracking.to_csv(data_path / 'train_player_tracking_ver2.csv', index=False)
test_player_tracking.to_csv(data_path / 'test_player_tracking_ver2.csv', index=False)

In [ ]:
tr_players_contact

,contact_id,game_play,datetime,step,nfl_player_id_1,nfl_player_id_2,contact
0,58168_003392_0_38590_43854,58168_003392,2020-09-11T03:01:48.100Z,0,38590,43854,0
1,58168_003392_0_38590_41257,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41257,0
2,58168_003392_0_38590_41944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41944,0
3,58168_003392_0_38590_42386,58168_003392,2020-09-11T03:01:48.100Z,0,38590,42386,0
4,58168_003392_0_38590_47944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,47944,0
...,...,...,...,...,...,...,...
4721591,58582_003121_91_47872_52493,58582_003121,2021-10-12T02:42:29.100Z,91,47872,52493,0
4721592,58582_003121_91_47872_52500,58582_003121,2021-10-12T02:42:29.100Z,91,47872,52500,0
4721593,58582_003121_91_47872_48220,58582_003121,2021-10-12T02:42:29.100Z,91,47872,48220,0
4721594,58582_003121_91_47872_47906,58582_003121,2021-10-12T02:42:29.100Z,91,47872,47906,0


In [ ]:
temp = tr_players_contact.loc[tr_players_contact.game_play == '58168_003392',
                              :].sort_values(by=['nfl_player_id_1', 'nfl_player_id_2'])
temp.sort_values(by=['step', 'nfl_player_id_1', 'nfl_player_id_2'])

,contact_id,game_play,datetime,step,nfl_player_id_1,nfl_player_id_2,contact
150,58168_003392_0_37084_37211,58168_003392,2020-09-11T03:01:48.100Z,0,37084,37211,0
149,58168_003392_0_37084_38556,58168_003392,2020-09-11T03:01:48.100Z,0,37084,38556,0
151,58168_003392_0_37084_38567,58168_003392,2020-09-11T03:01:48.100Z,0,37084,38567,0
140,58168_003392_0_37084_38590,58168_003392,2020-09-11T03:01:48.100Z,0,37084,38590,0
155,58168_003392_0_37084_39947,58168_003392,2020-09-11T03:01:48.100Z,0,37084,39947,0
...,...,...,...,...,...,...,...
17685,58168_003392_69_46522_47944,58168_003392,2020-09-11T03:01:55.000Z,69,46522,47944,0
17687,58168_003392_69_46522_52440,58168_003392,2020-09-11T03:01:55.000Z,69,46522,52440,0
17533,58168_003392_69_47837_47944,58168_003392,2020-09-11T03:01:55.000Z,69,47837,47944,0
17534,58168_003392_69_47837_52440,58168_003392,2020-09-11T03:01:55.000Z,69,47837,52440,0


In [ ]:
((tr_players_contact.groupby(['game_play']).count() / 231) - 1)

,contact_id,datetime,step,nfl_player_id_1,nfl_player_id_2,contact
game_play,,,,,,
58168_003392,69.0,69.0,69.0,69.0,69.0,69.0
58172_003247,125.0,125.0,125.0,125.0,125.0,125.0
58173_003606,129.0,129.0,129.0,129.0,129.0,129.0
58174_001792,79.0,79.0,79.0,79.0,79.0,79.0
58176_002844,58.0,58.0,58.0,58.0,58.0,58.0
...,...,...,...,...,...,...
58579_003527,81.0,81.0,81.0,81.0,81.0,81.0
58580_001136,72.0,72.0,72.0,72.0,72.0,72.0
58580_003294,65.0,65.0,65.0,65.0,65.0,65.0


In [133]:
def square(x):
    return x ** 2

def is_opposite(team1, team2):
    return 0 if team1 == team2 else 1

def relative_quantative(data1: pd.DataFrame, data2: pd.DataFrame) -> pd.DataFrame:
    # 동일 game_play, step에 대해 player1, player2에 대한 row of tracking data를 input으로 받는다.
    # position, velocitiy, acceleration
    # game_play, nfl_player_id, datetime, step
    # if isinstance(data1, pd.Series) and isinstance(data2, pd.Series):
    #     raise TypeError('Input data type must be Series.')
    # elif (data1.game_play != data2.game_play) or (data1.step != data2.step):
    #     raise ValueError('Time range between two dataframes does not match.')
    df1 = data1.reset_index(drop=True)
    df2 = data2.reset_index(drop=True)

    rel_df = df1.loc[:, ['game_play', 'step']]
    rel_df.loc[:, 'nfl_player_id_1'] = df1.nfl_player_id
    rel_df.loc[:, 'nfl_player_id_2'] = df2.nfl_player_id
    create_contact_id(rel_df, inplace=True)
    
    # 선수1과 선수2 사이의 변위, 상대속도, 상대가속도
    vector_quantative = ['x_position', 'y_position', 'x_vel', 'y_vel',
                         'x_acc', 'y_acc', 'direction', 'orientation']
    # 선수1과 선수2 사이의 거리, 상대속력, 상대가속력
    scalar_quantative = ['distance', 'speed', 'acceleration']

    # 선수1과 선수2 사이의 변위, 상대속도, 상대가속도 계산
    for idx, q in enumerate(vector_quantative):
        rel_df.loc[:, q] = df1.loc[:, q] - df2.loc[:, q]
        if (idx % 2 == 1) and (idx < 7):
            prev_q = vector_quantative[idx//2]
            scalar = scalar_quantative[idx//2]
            rel_df.loc[:, scalar] = rel_df.loc[:, q].apply(square) \
            + rel_df.loc[:, prev_q].apply(square)
            rel_df.loc[:, scalar] = rel_df.loc[:, scalar].apply(np.sqrt)

    team_df = pd.concat([df1.team, df2.team], axis=1)
    team_df.columns = ['team1', 'team2']

    rel_df.loc[:, 'opposite'] = team_df.apply(lambda x: is_opposite(x.team1, x.team2), axis=1)
    
    del team_df
    return rel_df

def ground_quantative(data: pd.DataFrame) -> pd.DataFrame:
    # 동일 game_play, step에 대해 player1, player2에 대한 row of tracking data를 input으로 받는다.
    # position, velocitiy, acceleration
    # game_play, nfl_player_id, datetime, step
    # if isinstance(data1, pd.Series) and isinstance(data2, pd.Series):
    #     raise TypeError('Input data type must be Series.')
    # elif (data1.game_play != data2.game_play) or (data1.step != data2.step):
    #     raise ValueError('Time range between two dataframes does not match.')
    df = data.reset_index(drop=True)
    res_df = df[['game_play', 'step']]
    res_df.loc[:, 'nfl_player_id_1'] = df.nfl_player_id
    res_df['nfl_player_id_2'] = 'G'
    create_contact_id(res_df, inplace=True)

    cols = ['x_position', 'y_position', 'distance',
            'x_vel', 'y_vel', 'speed',
            'x_acc', 'y_acc', 'acceleration',
            'direction', 'orientation']
    res_df.loc[:, cols] = df.loc[:, cols]
    res_df['opposite'] = 1

    return res_df

In [154]:
def filtering(quant_df, windows, id_cols, physics_cols):
    id_df = quant_df.loc[windows:, id_cols]
    if windows <= 1:
        rolling_df = quant_df.loc[:, physics_cols].diff()
    else:
        rolling_df = quant_df.loc[:, physics_cols].rolling(windows+1).mean()
    rolling_df = rolling_df.dropna()
    quant_df = pd.concat([id_df, rolling_df], axis=1)
    quant_df.reset_index(drop=True, inplace=True)

def make_graph_tracking_files(tracking, contact,
                              rolling: bool=False, 
                              windows: int=0, 
                              ground=False):
    final_ls = []

    id_cols = ['contact_id', 'game_play', 'step', 'nfl_player_id_1', 'nfl_player_id_2']

    if 'opposite' in tracking.columns.tolist():
        id_cols.append('opposite')

    physics_cols = ['x_position', 'y_position', 'distance',
                    'x_vel', 'y_vel', 'speed',
                    'x_acc', 'y_acc', 'acceleration',
                    'direction', 'orientation']
    
    for game_play in contact.game_play.unique():
        game_df = tracking.loc[tracking.game_play == game_play, :]
        
        # 필요한 frame만 뽑아내기 위함
        step_start = -windows if windows > 0 else 0
        game_df = game_df.loc[game_df.step >= step_start, :]
        
        # 비교군(추후 merge하기 위함)
        compare_df = contact.loc[contact.game_play == game_play, :]

        players1 = compare_df.nfl_player_id_1.unique()
        temp_ls = []

        for player1 in players1:
            player1_df = game_df.loc[game_df.nfl_player_id == player1, :]
            players2 = compare_df.loc[compare_df.nfl_player_id_1 == player1,
                                      'nfl_player_id_2'].unique()

            if ground:
                quant_df = ground_quantative(player1_df)

                if rolling:
                    filtering(quant_df, windows, id_cols, physics_cols)

                temp_ls.append(quant_df)
            else:
                #-------- only for contact between players --------#
                for player2 in players2:
                    player2_df = game_df.loc[game_df.nfl_player_id == player2, :]
                    quant_df = relative_quantative(player1_df, player2_df)
                    
                    if rolling:
                        filtering(quant_df, windows, id_cols, physics_cols)
                        
                    temp_ls.append(quant_df)

        temp_df = pd.concat(temp_ls, axis=0, ignore_index=True)
        result = pd.merge(compare_df, temp_df.loc[:, ['contact_id'] + physics_cols],
                          on='contact_id', how='inner')
        del temp_df
        final_ls.append(result)

    return pd.concat(final_ls, axis=0, ignore_index=True)

In [156]:
def make_train_dataset(windows):
    tr_filename1 = 'train_player_contact_tracking'
    tr_filename2 = 'train_ground_contact_tracking'
    te_filename1 = 'test_player_contact_tracking'
    te_filename2 = 'test_ground_contact_tracking'
    
    if windows == 0:
        tr_filename1 += '.csv'
        tr_filename2 += '.csv'
        te_filename1 += '.csv'
        te_filename2 += '.csv'
    else:
        tr_filename1 += f'_{windows}.csv'
        tr_filename2 += f'_{windows}.csv'
        te_filename1 += f'_{windows}.csv'
        te_filename2 += f'_{windows}.csv'
    
    rolling = True if windows > 0 else False

    tr_players_relative_tracking = make_graph_tracking_files(train_player_tracking,
                                                             tr_players_contact,
                                                             rolling=rolling,
                                                             windows=windows)
    
    tr_ground_relative_tracking = make_graph_tracking_files(train_player_tracking,
                                                            tr_ground_contact,
                                                            rolling=rolling,
                                                            windows=windows,
                                                            ground=True)
    
    te_players_relative_tracking = make_graph_tracking_files(test_player_tracking,
                                                             te_players_contact,
                                                             rolling=rolling,
                                                             windows=windows)
    
    te_ground_relative_tracking = make_graph_tracking_files(test_player_tracking,
                                                             te_ground_contact,
                                                             rolling=rolling,
                                                             windows=windows,
                                                             ground=True)

    # save files
    tr_players_relative_tracking.to_csv(data_path / tr_filename1, index=False)
    te_players_relative_tracking.to_csv(data_path / te_filename1, index=False)
    tr_ground_relative_tracking.to_csv(data_path / tr_filename2, index=False)
    te_ground_relative_tracking.to_csv(data_path / te_filename2, index=False)

In [158]:
import time

for i in range(5):
    start = time.time()
    a = 3
    b = 5
    end = time.time()
    print(end - start)

1.6689300537109375e-06
2.1457672119140625e-06
4.76837158203125e-07
4.76837158203125e-07
2.384185791015625e-07


In [ ]:
from tqdm.notebook import tqdm

for window in tqdm(range(11)):
    make_train_dataset(window)

  0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
temp = test_player_tracking
temp.loc[:, 'opposite'] = 1
temp

,game_play,game_key,play_id,nfl_player_id,datetime,step,team,position,jersey_number,x_position,...,distance,direction,orientation,acceleration,sa,x_vel,y_vel,x_acc,y_acc,opposite
0,58172_003247,58172,3247,41937,2020-09-13T19:30:20.200Z,-272,home,MLB,57,58.777632,...,0.420624,69.26,77.57,2.862072,2.743200,1.560786,4.121795,0.208063,2.854499,1
1,58172_003247,58172,3247,45345,2020-09-13T19:30:20.200Z,-272,away,ILB,50,69.521832,...,0.292608,270.48,265.09,1.709928,-1.655064,0.024130,-2.880259,-0.443521,1.651406,1
2,58172_003247,58172,3247,46205,2020-09-13T19:30:20.200Z,-272,home,DE,98,60.021216,...,0.182880,83.27,110.15,0.630936,-0.621792,0.192887,1.634579,-0.179159,-0.604965,1
3,58172_003247,58172,3247,43406,2020-09-13T19:30:20.200Z,-272,home,WR,14,33.238440,...,0.164592,119.50,105.97,0.566928,0.356616,-0.814993,1.440494,0.207975,0.527403,1
4,58172_003247,58172,3247,48233,2020-09-13T19:30:20.200Z,-272,away,RB,45,69.585840,...,0.310896,283.41,269.44,0.804672,-0.795528,0.712540,-2.988617,-0.066831,0.801892,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14867,58168_003392,58168,3392,43395,2020-09-11T03:01:57.300Z,92,away,FS,23,46.661832,...,0.155448,68.99,71.68,0.320040,-0.182880,0.557329,1.451135,-0.310749,-0.076557,1
14868,58168_003392,58168,3392,39947,2020-09-11T03:01:57.300Z,92,home,T,72,39.072312,...,0.118872,118.78,117.03,0.530352,-0.512064,-0.523881,0.953724,0.125516,-0.515285,1
14869,58168_003392,58168,3392,44822,2020-09-11T03:01:57.300Z,92,home,QB,15,32.013144,...,0.137160,284.78,287.34,0.347472,-0.064008,0.345241,-1.308534,-0.346554,-0.025236,1
14870,58168_003392,58168,3392,38590,2020-09-11T03:01:57.300Z,92,home,G,70,39.977568,...,0.146304,108.92,114.00,0.374904,-0.374904,-0.456598,1.332095,0.121562,-0.354649,1


In [ ]:
tr_ground_contact_tracking = make_graph_tracking_files(train_player_tracking,
                                                       tr_ground_contact,
                                                       ground=True)
te_ground_contact_tracking = make_graph_tracking_files(test_player_tracking,
                                                       te_ground_contact,
                                                       ground=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
<ipython-input-133-f4847df6916f>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_df['nfl_player_id_2'] = 'G'
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [ ]:
tr_ground_contact_tracking

,contact_id,game_play,datetime,step,nfl_player_id_1,nfl_player_id_2,contact,x_position,y_position,distance,x_vel,y_vel,speed,x_acc,y_acc,acceleration
0,58168_003392_0_38590_43854,58168_003392,2020-09-11T03:01:48.100Z,0,38590,43854,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,58168_003392_0_38590_41257,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41257,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58168_003392_0_38590_41944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41944,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,58168_003392_0_38590_42386,58168_003392,2020-09-11T03:01:48.100Z,0,38590,42386,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,58168_003392_0_38590_47944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,47944,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4310980,58582_003121_91_47872_52493,58582_003121,2021-10-12T02:42:29.100Z,91,47872,52493,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4310981,58582_003121_91_47872_52500,58582_003121,2021-10-12T02:42:29.100Z,91,47872,52500,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4310982,58582_003121_91_47872_48220,58582_003121,2021-10-12T02:42:29.100Z,91,47872,48220,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4310983,58582_003121_91_47872_47906,58582_003121,2021-10-12T02:42:29.100Z,91,47872,47906,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
te_ground_contact_tracking

,contact_id,game_play,step,nfl_player_id_1,nfl_player_id_2,contact,x_position,y_position,distance,x_vel,y_vel,speed,x_acc,y_acc,acceleration
0,58168_003392_0_38590_43854,58168_003392,0,38590,43854,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,58168_003392_0_38590_41257,58168_003392,0,38590,41257,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58168_003392_0_38590_41944,58168_003392,0,38590,41944,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,58168_003392_0_38590_42386,58168_003392,0,38590,42386,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,58168_003392_0_38590_47944,58168_003392,0,38590,47944,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45271,58172_003247_125_47912_48241,58172_003247,125,47912,48241,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45272,58172_003247_125_47912_47920,58172_003247,125,47912,47920,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45273,58172_003247_125_47912_48335,58172_003247,125,47912,48335,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45274,58172_003247_125_47912_52521,58172_003247,125,47912,52521,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
tr_ground_contact_tracking.to_csv(data_path / 'train_ground_contact_tracking.csv', index=False)
te_ground_contact_tracking.to_csv(data_path / 'test_ground_contact_tracking.csv', index=False)

In [ ]:
tr_players_relative_tracking

,game_play,step,nfl_player_id_1,nfl_player_id_2,x_position,y_position,distance,x_vel,y_vel,speed,x_acc,y_acc,acceleration
0,58580_001136,-108,44830,G,56.317896,38.953440,0.100584,-0.647930,0.781268,1.014984,0.157213,-0.629901,0.649224
1,58580_001136,-108,40031,G,64.492632,20.144232,0.027432,-0.281719,0.079082,0.292608,-0.256028,0.001435,0.256032
2,58580_001136,-108,43351,G,58.878216,9.957816,0.073152,0.556335,-0.474987,0.731520,-0.305984,0.261243,0.402336
3,58580_001136,-108,47800,G,54.388512,24.515064,0.009144,-0.047882,0.204789,0.210312,0.635509,0.993745,1.179576
4,58580_001136,-108,41242,G,52.422552,22.677120,0.009144,-0.014521,0.023274,0.027432,-0.033881,0.054306,0.064008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353048,58575_003081,140,53435,G,86.904576,31.757112,0.128016,-0.398353,-1.216604,1.280160,0.576321,-0.179083,0.603504
1353049,58575_003081,140,52947,G,104.424480,15.343632,0.402336,-1.668460,-3.661102,4.023360,0.406130,0.066135,0.411480
1353050,58575_003081,140,42508,G,74.971656,8.430768,0.265176,1.766783,-2.001908,2.670048,-0.060119,0.434775,0.438912
1353051,58575_003081,140,43854,G,70.582536,51.270408,0.109728,-1.014462,0.506455,1.133856,-0.464376,-0.766402,0.896112


In [ ]:
tr_ground_contact_tracking

,game_play,step,nfl_player_id_1,nfl_player_id_2,x_position,y_position,distance,x_vel,y_vel,speed,x_acc,y_acc,acceleration
0,58172_003247,-272,41937,G,58.777632,10.323576,0.420624,1.560786,4.121795,4.407408,0.208063,2.854499,2.862072
1,58172_003247,-272,47889,G,69.695568,35.606736,0.274320,0.845365,-2.513402,2.651760,-0.588220,0.882661,1.060704
2,58172_003247,-272,52852,G,58.192416,13.459968,0.210312,0.142779,2.006607,2.011680,-0.091509,-1.286052,1.289304
3,58172_003247,-272,48241,G,67.610736,35.743896,0.365760,0.967685,-3.403848,3.538728,-0.465089,1.635958,1.700784
4,58172_003247,-272,39008,G,61.356240,17.099280,0.164592,0.193384,1.643727,1.655064,-0.242747,-0.146375,0.283464
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14867,58168_003392,92,39947,G,39.072312,16.934688,0.118872,-0.523881,0.953724,1.088136,0.125516,-0.515285,0.530352
14868,58168_003392,92,45695,G,35.762184,10.579608,0.155448,0.025278,1.508548,1.508760,0.119238,-0.267211,0.292608
14869,58168_003392,92,42565,G,38.084760,13.359384,0.146304,-0.421337,1.343665,1.408176,0.260215,-0.330473,0.420624
14870,58168_003392,92,38556,G,36.740592,15.855696,0.137160,0.367362,1.330977,1.380744,-0.072986,-0.264432,0.274320


## Load Preprocessing Data

In [ ]:
train_player_tracking = pd.read_csv(data_path / 'train_player_tracking_ver2.csv')
test_player_tracking = pd.read_csv(data_path / 'test_player_tracking_ver2.csv')

tr_players_relative_tracking = pd.read_csv(data_path / 'train_player_contact_tracking.csv')
te_players_relative_tracking = pd.read_csv(data_path / 'test_player_contact_tracking.csv')

In [ ]:
test_player_tracking

,game_key,play_id,nfl_player_id,datetime,step,team,position,jersey_number,x_position,y_position,speed,distance,direction,orientation,acceleration,sa,x_vel,y_vel,x_acc,y_acc
game_play,,,,,,,,,,,,,,,,,,,,
58172_003247,58172,3247,41937,2020-09-13T19:30:20.200Z,-272,home,MLB,57,58.777632,10.323576,4.407408,0.420624,69.26,77.57,2.862072,2.743200,1.560786,4.121795,0.208063,2.854499
58172_003247,58172,3247,45345,2020-09-13T19:30:20.200Z,-272,away,ILB,50,69.521832,33.595056,2.880360,0.292608,270.48,265.09,1.709928,-1.655064,0.024130,-2.880259,-0.443521,1.651406
58172_003247,58172,3247,46205,2020-09-13T19:30:20.200Z,-272,home,DE,98,60.021216,14.392656,1.645920,0.182880,83.27,110.15,0.630936,-0.621792,0.192887,1.634579,-0.179159,-0.604965
58172_003247,58172,3247,43406,2020-09-13T19:30:20.200Z,-272,home,WR,14,33.238440,13.661136,1.655064,0.164592,119.50,105.97,0.566928,0.356616,-0.814993,1.440494,0.207975,0.527403
58172_003247,58172,3247,48233,2020-09-13T19:30:20.200Z,-272,away,RB,45,69.585840,29.397960,3.072384,0.310896,283.41,269.44,0.804672,-0.795528,0.712540,-2.988617,-0.066831,0.801892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58168_003392,58168,3392,43395,2020-09-11T03:01:57.300Z,92,away,FS,23,46.661832,12.463272,1.554480,0.155448,68.99,71.68,0.320040,-0.182880,0.557329,1.451135,-0.310749,-0.076557
58168_003392,58168,3392,39947,2020-09-11T03:01:57.300Z,92,home,T,72,39.072312,16.934688,1.088136,0.118872,118.78,117.03,0.530352,-0.512064,-0.523881,0.953724,0.125516,-0.515285
58168_003392,58168,3392,44822,2020-09-11T03:01:57.300Z,92,home,QB,15,32.013144,17.080992,1.353312,0.137160,284.78,287.34,0.347472,-0.064008,0.345241,-1.308534,-0.346554,-0.025236


In [ ]:
def inplace(func):
    def wrapper(data, inplace: bool=False):                           # 호출할 함수를 감싸는 함수
        if inplace:
            func(data, inplace)
        else:
            df = data.copy()
            func(df, inplace)
            return df
    return wrapper                           # wrapper 함수 반환

## contact id 만드는 함수
@inplace
def create_contact_id(df, inplace: bool=False):
    # Create contact ids
    cols = list(df)
    df.loc[:, 'contact_id'] = (
        df.loc[:, "game_play"]
        + "_"
        + df.loc[:, "step"].astype("str")
        + "_"
        + df.loc[:, "nfl_player_id_1"].astype("str")
        + "_"
        + df.loc[:, "nfl_player_id_2"].astype("str")
    )

    temp = df[cols]
    df.drop(columns=cols, inplace=True)
    df[cols] = temp
    del temp
    del cols
    return df

In [ ]:
create_contact_id(tr_players_relative_tracking, inplace=True)
tr_players_relative_tracking

,contact_id,game_play,step,nfl_player_id_1,nfl_player_id_2,x_position,y_position,dist,x_vel,y_vel,speed,x_acc,y_acc,acc
0,58580_001136_-108_44830_47800,58580_001136,-108,44830,47800,1.929384,14.438376,14.566716,-0.600048,0.576479,14.449880,-0.478295,-1.623646,1.730978
1,58580_001136_-108_47800_41242,58580_001136,-108,47800,41242,1.965960,1.837944,2.691289,-0.033361,0.181515,1.846885,0.669390,0.939439,0.940031
2,58580_001136_-108_53876_43351,58580_001136,-108,53876,43351,-4.334256,10.808208,11.644876,-0.766592,0.470179,10.818430,0.516241,-0.256435,0.808345
3,58580_001136_-108_53876_37082,58580_001136,-108,53876,37082,2.276856,-6.245352,6.647443,-0.272770,0.085371,6.245935,0.257113,0.048415,0.277034
4,58580_001136_-108_46206_41242,58580_001136,-108,46206,41242,0.036576,-2.450592,2.450865,0.281671,-0.230871,2.461443,0.311635,-0.620815,0.681726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14205529,58575_003081_140_43757_46699,58575_003081,140,43757,46699,3.063240,12.527280,12.896363,0.373479,0.045966,12.527364,0.309698,-0.550557,0.665282
14205530,58575_003081_140_46915_41356,58575_003081,140,46915,41356,1.499616,-6.684264,6.850418,-0.621484,-0.228331,6.688163,0.904338,-0.022229,0.621881
14205531,58575_003081_140_44174_46915,58575_003081,140,44174,46915,-33.448752,35.734752,48.946823,1.637078,4.457448,36.011683,-1.177796,0.086989,1.639387
14205532,58575_003081_140_43757_46915,58575_003081,140,43757,46915,-0.512064,6.748272,6.767672,-0.134321,1.952215,7.024978,-0.321784,-0.199730,0.240695


In [ ]:
tr_players_contact

,contact_id,game_play,datetime,step,nfl_player_id_1,nfl_player_id_2,contact
0,58168_003392_0_38590_43854,58168_003392,2020-09-11T03:01:48.100Z,0,38590,43854,0
1,58168_003392_0_38590_41257,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41257,0
2,58168_003392_0_38590_41944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41944,0
3,58168_003392_0_38590_42386,58168_003392,2020-09-11T03:01:48.100Z,0,38590,42386,0
4,58168_003392_0_38590_47944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,47944,0
...,...,...,...,...,...,...,...
4721591,58582_003121_91_47872_52493,58582_003121,2021-10-12T02:42:29.100Z,91,47872,52493,0
4721592,58582_003121_91_47872_52500,58582_003121,2021-10-12T02:42:29.100Z,91,47872,52500,0
4721593,58582_003121_91_47872_48220,58582_003121,2021-10-12T02:42:29.100Z,91,47872,48220,0
4721594,58582_003121_91_47872_47906,58582_003121,2021-10-12T02:42:29.100Z,91,47872,47906,0


In [ ]:
tr_players_relative_tracking

,game_play,step,nfl_player_id_1,nfl_player_id_2,x_position,y_position,dist,x_vel,y_vel,speed,x_acc,y_acc,acc
0,58580_001136,-108,44830,47800,1.929384,14.438376,14.566716,-0.600048,0.576479,14.449880,-0.478295,-1.623646,1.730978
1,58580_001136,-108,47800,41242,1.965960,1.837944,2.691289,-0.033361,0.181515,1.846885,0.669390,0.939439,0.940031
2,58580_001136,-108,53876,43351,-4.334256,10.808208,11.644876,-0.766592,0.470179,10.818430,0.516241,-0.256435,0.808345
3,58580_001136,-108,53876,37082,2.276856,-6.245352,6.647443,-0.272770,0.085371,6.245935,0.257113,0.048415,0.277034
4,58580_001136,-108,46206,41242,0.036576,-2.450592,2.450865,0.281671,-0.230871,2.461443,0.311635,-0.620815,0.681726
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14205529,58575_003081,140,43757,46699,3.063240,12.527280,12.896363,0.373479,0.045966,12.527364,0.309698,-0.550557,0.665282
14205530,58575_003081,140,46915,41356,1.499616,-6.684264,6.850418,-0.621484,-0.228331,6.688163,0.904338,-0.022229,0.621881
14205531,58575_003081,140,44174,46915,-33.448752,35.734752,48.946823,1.637078,4.457448,36.011683,-1.177796,0.086989,1.639387
14205532,58575_003081,140,43757,46915,-0.512064,6.748272,6.767672,-0.134321,1.952215,7.024978,-0.321784,-0.199730,0.240695


In [ ]:
tr_players_relative_tracking

,game_play,step,nfl_player_id_1,nfl_player_id_2,x_position,y_position,distance,x_vel,y_vel,speed,x_acc,y_acc,acceleration
0,58580_001136,-108,44830,G,56.317896,38.953440,0.100584,-0.647930,0.781268,1.014984,0.157213,-0.629901,0.649224
1,58580_001136,-108,40031,G,64.492632,20.144232,0.027432,-0.281719,0.079082,0.292608,-0.256028,0.001435,0.256032
2,58580_001136,-108,43351,G,58.878216,9.957816,0.073152,0.556335,-0.474987,0.731520,-0.305984,0.261243,0.402336
3,58580_001136,-108,47800,G,54.388512,24.515064,0.009144,-0.047882,0.204789,0.210312,0.635509,0.993745,1.179576
4,58580_001136,-108,41242,G,52.422552,22.677120,0.009144,-0.014521,0.023274,0.027432,-0.033881,0.054306,0.064008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353048,58575_003081,140,53435,G,86.904576,31.757112,0.128016,-0.398353,-1.216604,1.280160,0.576321,-0.179083,0.603504
1353049,58575_003081,140,52947,G,104.424480,15.343632,0.402336,-1.668460,-3.661102,4.023360,0.406130,0.066135,0.411480
1353050,58575_003081,140,42508,G,74.971656,8.430768,0.265176,1.766783,-2.001908,2.670048,-0.060119,0.434775,0.438912
1353051,58575_003081,140,43854,G,70.582536,51.270408,0.109728,-1.014462,0.506455,1.133856,-0.464376,-0.766402,0.896112


In [ ]:
key_columns = ['game_play', 'step', 'nfl_player_id_1', 'nfl_player_id_2']

train_players = pd.merge(tr_players_contact,
                         tr_players_relative_tracking,
                         on=key_columns,
                         how='left'
                         )
train_players

ValueError: ignored

In [ ]:
tr_players_contact

,contact_id,game_play,datetime,step,nfl_player_id_1,nfl_player_id_2,contact
0,58168_003392_0_38590_43854,58168_003392,2020-09-11T03:01:48.100Z,0,38590,43854,0
1,58168_003392_0_38590_41257,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41257,0
2,58168_003392_0_38590_41944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41944,0
3,58168_003392_0_38590_42386,58168_003392,2020-09-11T03:01:48.100Z,0,38590,42386,0
4,58168_003392_0_38590_47944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,47944,0
...,...,...,...,...,...,...,...
4721591,58582_003121_91_47872_52493,58582_003121,2021-10-12T02:42:29.100Z,91,47872,52493,0
4721592,58582_003121_91_47872_52500,58582_003121,2021-10-12T02:42:29.100Z,91,47872,52500,0
4721593,58582_003121_91_47872_48220,58582_003121,2021-10-12T02:42:29.100Z,91,47872,48220,0
4721594,58582_003121_91_47872_47906,58582_003121,2021-10-12T02:42:29.100Z,91,47872,47906,0


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef

from lightgbm.sklearn import LGBMClassifier
from xgboost.sklearn import XGBClassifier

In [ ]:
left_columns = ['contact_id', 'game_play', 'step', 'nfl_player_id_1']
right_columns = ['contact_id', 'game_play', 'step', 'nfl_player_id']

train_ground = pd.merge(tr_ground_contact,
                        train_player_tracking,
                        left_on=left_columns,
                        right_on=right_columns)
train_ground

In [ ]:
test_players_contact_tracking = make_graph_tracking_files(test_player_tracking)

In [ ]:
test_players_contact_tracking

,game_play,step,nfl_player_id_1,nfl_player_id_2,x_position,y_position,dist,x_vel,y_vel,speed,x_acc,y_acc,acc
0,58172_003247,-272,41937,45345,-10.744200,-23.271480,25.632004,1.536656,7.002054,24.302069,0.651584,1.203093,1.951600
1,58172_003247,-272,48335,52939,19.238976,12.655296,23.028129,-0.233288,-4.122983,13.309978,0.418484,0.938521,0.967081
2,58172_003247,-272,46273,46451,-11.841480,-21.433536,24.487081,0.326813,2.897197,21.628458,0.583828,-1.203993,1.247559
3,58172_003247,-272,48233,52521,7.424928,7.415784,10.493970,0.908125,-2.417685,7.799939,-0.290556,-0.291012,0.953614
4,58172_003247,-272,41937,52521,-3.383280,-11.658600,12.139586,1.756371,4.692727,12.567603,-0.015662,1.761595,2.487580
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156151,58168_003392,92,41257,38567,-1.170432,-3.255264,3.459285,1.594413,0.273210,3.266709,-0.551363,0.039398,1.594900
156152,58168_003392,92,38567,39947,-1.088136,-1.847088,2.143776,-0.070025,-0.370909,1.883961,0.148223,0.533130,0.537709
156153,58168_003392,92,41257,37211,0.073152,5.824728,5.825187,1.055615,1.447825,6.001971,0.481301,0.050856,1.056840
156154,58168_003392,92,38567,37084,2.916936,3.063240,4.229888,-1.330058,-0.574495,3.116646,0.466553,0.502076,1.421666
